## Scraping Multiple Pages - Miami

#### Importing Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import pandas as pd

#### Scraping the data

In [2]:
address = []
bedrooms = []
bathrooms = []
sqft = []
year_built = []
parking = []
pool = []
price = []

# Url part I
url_1 = 'https://www.trulia.com'

for i in range(1, 26):
    
    #website
    www = 'https://www.trulia.com/FL/Miami/' +str(i) +'_p/'
    
    #requests
    response = requests.get(www)
    
    # soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #result container
    result_container = soup.find_all('li', {'class':'SearchResultsList__WideCell-b7y9ki-2'})
    
    #update results
    results_update = []
    
    # only results with attribute 'data-testeid'
    for r in result_container:
        if r.has_attr('data-testid'):
            results_update.append(r)
        
    #relative url
    relative_url = []
    
    #loop through results
    for i in results_update:
        for j in i.find_all('div', {'data-testid':'property-card-details'}):
            relative_url.append(j.find('a').get('href'))
            
    #empty list url_joined
    url_joined = []

    for i in relative_url:
        url_joined.append(urllib.parse.urljoin(url_1, i))
        
       
    # loop through all joined links
    for i in url_joined:
        response = requests.get(i)

        #create soup object
        soup = BeautifulSoup(response.content, 'html.parser')

        #address
        try:
            address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
        except:
            address.append('')

        #bedrooms
        try:
            bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
        except:
            bedrooms.append('')

        #bathrooms
        try:
            bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
        except:
            bathrooms.append('')

        #sqft
        try:
            sqft.append(soup.find('li', {'data-testid':'floor'}).get_text())
        except:
            sqft.append('')

        #year built
        try:
            year_built.append(soup.find('div', string='Year Built').findNext('div').get_text())
        except:
            year_built.append('')

         #parking
        try:
            parking.append(soup.find('div', string='Parking').findNext('div').get_text())
        except:
            parking.append('')

         #pool
        try:
            pool.append(soup.find('div', string='Pool').findNext('div').get_text())
        except:
            pool.append('')

         #price
        try:
            price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
        except:
            price.append('')

        #creating a dictionary with the results
        output = {'Address': address, 'Bedrooms':bedrooms, 'Bathrooms': bathrooms, 'Area': sqft, 'Year Built': year_built,
                 'Parking': parking, 'Pool': pool, 'Price': price}

In [3]:
df = pd.DataFrame(output)
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price
0,1133 Belle Meade Island Dr,5 Beds,6 Baths,"8,200 sqft (on 0.30 acres)",2021,8 Car Garage,No,"$14,900,000"
1,1776 SW 16th Ter,3 Beds,2 Baths,"1,866 sqft",1925,Garage,No,"$750,000"
2,3701 Battersea Rd,4 Beds,4 Baths,"3,172 sqft",1958,Garage,No,"$2,745,000"
3,2965 SW 25th Ter,3 Beds,3 Baths,"1,626 sqft",1981,Garage,No,"$499,000"
4,2421 Inagua Ave,3 Beds,2 Baths,"1,485 sqft",1952,Open Parking,No,"$1,500,000"
...,...,...,...,...,...,...,...,...
1025,3012 Kirk St,3 Beds,4 Baths,"4,299 sqft",2019,Open Parking,No,"$3,990,000"
1026,1374 S Venetian Way,7 Beds,9 Baths,"7,886 sqft",2006,3 Car Garage,No,"$21,995,000"
1027,757 NE 74th St,3 Beds,2 Baths,"1,579 sqft",1939,Open Parking,No,"$950,000"
1028,277 NW 43rd St,3 Beds,3 Baths,"1,999 sqft",2021,Open Parking,No,"$1,299,500"


In [4]:
df['Location'] = 'Miami'
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price,Location
0,1133 Belle Meade Island Dr,5 Beds,6 Baths,"8,200 sqft (on 0.30 acres)",2021,8 Car Garage,No,"$14,900,000",Miami
1,1776 SW 16th Ter,3 Beds,2 Baths,"1,866 sqft",1925,Garage,No,"$750,000",Miami
2,3701 Battersea Rd,4 Beds,4 Baths,"3,172 sqft",1958,Garage,No,"$2,745,000",Miami
3,2965 SW 25th Ter,3 Beds,3 Baths,"1,626 sqft",1981,Garage,No,"$499,000",Miami
4,2421 Inagua Ave,3 Beds,2 Baths,"1,485 sqft",1952,Open Parking,No,"$1,500,000",Miami
...,...,...,...,...,...,...,...,...,...
1025,3012 Kirk St,3 Beds,4 Baths,"4,299 sqft",2019,Open Parking,No,"$3,990,000",Miami
1026,1374 S Venetian Way,7 Beds,9 Baths,"7,886 sqft",2006,3 Car Garage,No,"$21,995,000",Miami
1027,757 NE 74th St,3 Beds,2 Baths,"1,579 sqft",1939,Open Parking,No,"$950,000",Miami
1028,277 NW 43rd St,3 Beds,3 Baths,"1,999 sqft",2021,Open Parking,No,"$1,299,500",Miami


In [5]:
df.to_csv('df_miami.csv', index=False)